In [1]:
import pandas as pd
import datetime
import os
import numpy as np
import gc
# Sales>1 per item/subcalss line
# Sales >7000 ids removed, 1196 ids out of 17883899 rewards shoppers in total
# 11.7% Unknown (not valid) ids, they are included in the P/S/T cumulative sorting

In [2]:
def recursive_file_gen(root_path):
    for root,dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

# Master_file_id_zip

In [3]:
# Conbined before 2019
df_id_zip=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',
                          usecols=['customer_id_hashed','customer_zip_code'],dtype=str)
print(df_id_zip.shape)

(35650450, 2)


In [4]:
# Lapsed
df_id_zip_1 = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     dtype = 'str',sep = '|',usecols = ['customer_id_hashed','customer_zip_code'])

df_id_zip=df_id_zip.append(df_id_zip_1).drop_duplicates("customer_id_hashed")
del df_id_zip_1
print(df_id_zip.shape)
gc.collect()

(23907550, 2)


14

In [5]:
# 2018Q1 to 2019Q1 new sign ups
df_id_zip_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                         compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','customer_zip_code'])
df_id_zip=df_id_zip_1.append(df_id_zip).drop_duplicates("customer_id_hashed")
print(df_id_zip.shape)

(27890080, 2)


In [6]:
# The gap
df_id_zip_1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                         compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','customer_zip_code'])
df_id_zip=df_id_zip_1.append(df_id_zip).drop_duplicates("customer_id_hashed")
print(df_id_zip.shape)

(28351272, 2)


In [7]:
new_sign_ups_by_week_file_list=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
new_sign_ups_by_week_file_list=[x for x in new_sign_ups_by_week_file_list if "aster" in x]
new_sign_ups_by_week_file_df=pd.DataFrame({"file_path":new_sign_ups_by_week_file_list})
new_sign_ups_by_week_file_df['week_end_dt']=new_sign_ups_by_week_file_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
new_sign_ups_by_week_file_df=new_sign_ups_by_week_file_df[new_sign_ups_by_week_file_df['week_end_dt']>"2019-06-01"] #> as the max date above

df_id_zip_1=pd.DataFrame()
for file in new_sign_ups_by_week_file_df['file_path'].unique().tolist():
    df=pd.read_table(file,dtype=str,usecols=['customer_id_hashed','customer_zip_code'],sep="|")
    df_id_zip_1=df_id_zip_1.append(df)
    
df_id_zip=df_id_zip.append(df_id_zip_1)
df_id_zip=df_id_zip.drop_duplicates("customer_id_hashed")
del df_id_zip_1
gc.collect()

print(df_id_zip.shape)

(28703956, 2)


In [8]:
df_id_zip=df_id_zip[pd.notnull(df_id_zip['customer_zip_code'])]
df_id_zip['customer_zip_code']=df_id_zip['customer_zip_code'].astype(str)
df_id_zip['customer_zip_code']=df_id_zip['customer_zip_code'].apply(lambda x: x.split("-")[0].zfill(5)[:5])


In [9]:
###### Matching with Nielsen data

DMA_nielsen=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,usecols=[0,2,6],skiprows=1)
DMA_nielsen.columns=['customer_zip_code','DMA_set','ST_set']
DMA_nielsen=DMA_nielsen.drop_duplicates()
def func_set_str(x):
    return str(set(x))
DMA_nielsen=DMA_nielsen.groupby("customer_zip_code")['DMA_set','ST_set'].agg(func_set_str).reset_index()
DMA_nielsen.head(3)

,customer_zip_code,DMA_set,ST_set
0,00501,{'NEW YORK'},{'NY'}
1,00544,{'NEW YORK'},{'NY'}
2,01001,{'SPRINGFIELD-HOLYOKE'},{'MA'}


In [10]:
df_id_zip=pd.merge(df_id_zip,DMA_nielsen,on="customer_zip_code",how="left")

print(df_id_zip.shape)
print(df_id_zip['customer_id_hashed'].nunique())
print(df_id_zip[pd.isnull(df_id_zip['DMA_set'])].shape)


(25686877, 4)
25686877
(187456, 4)


# Transactions_1_Year

In [11]:
item_level_sales_historical=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
item_level_sales_historica=[x for x in item_level_sales_historical if (("MediaStormDailySalesHistory" in x) & (".txt" in x))]
df_item_level_sales_historical=pd.DataFrame({"file_path":item_level_sales_historica})
df_item_level_sales_historical['week_end_dt']=df_item_level_sales_historical['file_path'].apply(lambda x: x.split("/MediaStormDailySalesHistory")[1][:8])
print(df_item_level_sales_historical.shape)
print(df_item_level_sales_historical['file_path'].nunique())
print(df_item_level_sales_historical['week_end_dt'].max())
print(df_item_level_sales_historical['week_end_dt'].min())


(27, 2)
27
20190209
20180811


In [12]:
this_year_item_level=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
this_year_item_level=[x for x in this_year_item_level if (("MediaStormDailySales" in x) & (".txt" in x))]
df_item_level_sales_2019=pd.DataFrame({"file_path":this_year_item_level})
df_item_level_sales_2019['week_end_dt']=df_item_level_sales_2019['file_path'].apply(lambda x: x.split("2019_by_weeks/MediaStorm_")[1][:10].replace("-",""))


df_item_level_sales_2019=df_item_level_sales_2019[df_item_level_sales_2019['week_end_dt']>df_item_level_sales_historical['week_end_dt'].max()]
print(df_item_level_sales_2019.shape)
print(df_item_level_sales_2019['file_path'].nunique())
print(df_item_level_sales_2019['week_end_dt'].max())
print(df_item_level_sales_2019['week_end_dt'].min())


(19, 2)
19
20190622
20190216


In [13]:
last_year_subclass_level=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
last_year_subclass_level=[x for x in last_year_subclass_level if (("MediaStormDailySales" in x) & (".txt" in x))]
df_subclass_level_sales_2018=pd.DataFrame({"file_path":last_year_subclass_level})
df_subclass_level_sales_2018['week_end_dt']=df_subclass_level_sales_2018['file_path'].apply(lambda x: x.split("2018_by_weeks/MediaStorm_")[1][:10].replace("-",""))


df_subclass_level_sales_2018=df_subclass_level_sales_2018[df_subclass_level_sales_2018['week_end_dt']<df_item_level_sales_historical['week_end_dt'].min()]
print(df_subclass_level_sales_2018.shape)
print(df_subclass_level_sales_2018['file_path'].nunique())
print(df_subclass_level_sales_2018['week_end_dt'].max())
print(df_subclass_level_sales_2018['week_end_dt'].min())

(8, 2)
8
20180804
20180616


In [14]:
all_df_item_level=df_item_level_sales_2019.append(df_item_level_sales_historical).sort_values("week_end_dt")
print(all_df_item_level.shape)
print(all_df_item_level['file_path'].nunique())

(46, 2)
46


In [15]:
df_subclass_level_sales_2018=df_subclass_level_sales_2018.sort_values("week_end_dt").tail(52-all_df_item_level['file_path'].nunique())
df_subclass_level_sales_2018=df_subclass_level_sales_2018.reset_index()
del df_subclass_level_sales_2018['index']

In [16]:
# Sales>1
df_sales_all=pd.DataFrame(columns=['location_id','transaction_dt','customer_id_hashed','sales'])
for file in df_subclass_level_sales_2018['file_path'].unique().tolist():
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','subclass_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df=df[df['subclass_transaction_amt']>1]
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['subclass_transaction_amt'].sum().to_frame().reset_index()
    df_trans=df[['location_id','transaction_dt','customer_id_hashed','transaction_id']].drop_duplicates()
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt','customer_id_hashed'],how="outer")
    df=df.rename(columns={"subclass_transaction_amt":"sales","transaction_id":"trans"})
    df_sales_all=df_sales_all.append(df)

In [17]:
error_file_list=[]
for file in all_df_item_level['file_path'].unique().tolist():    
    try:
        if ".zip" in file:
            df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','customer_id_hashed','transaction_id','item_transaction_amt'],compression="zip")
        elif ".txt" in file:
            df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','customer_id_hashed','transaction_id','item_transaction_amt'])
    except:
        error_file_list=error_file_list+[file]
        df=pd.DataFrame(columns=['location_id','transaction_dt','customer_id_hashed','item_transaction_amt','transaction_id'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df[df['item_transaction_amt']>1]
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
    df_trans=df[['location_id','transaction_dt','customer_id_hashed','transaction_id']].drop_duplicates()
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt','customer_id_hashed'],how="outer")
    df=df.rename(columns={"item_transaction_amt":"sales","transaction_id":"trans"})
    df_sales_all=df_sales_all.append(df)

KeyboardInterrupt: 

In [ ]:
func_date_store={"transaction_dt":['nunique','max','min']}
df_summary_store_date=df_sales_all.groupby(['location_id'])['transaction_dt'].agg(func_date_store).reset_index()
new_cols=df_summary_store_date.columns.tolist()
new_col_list=[]
for col in new_cols:
    if col[1]=="":
        new_col=col[0]
    else:
        new_col=col[0]+"_"+col[1]
    new_col_list=new_col_list+[new_col]
df_summary_store_date.columns=new_col_list

In [ ]:
store_list_fewer_than_2_months=df_summary_store_date[df_summary_store_date['transaction_dt_nunique']<=60]
store_list_fewer_2_to_12_months=df_summary_store_date[df_summary_store_date['transaction_dt_nunique']>60]
print("store_list_fewer_2_to_12_months['location_id'].nunique(): " + str((store_list_fewer_2_to_12_months['location_id'].nunique())))
print("store_list_fewer_than_2_months['location_id'].nunique(): " + str((store_list_fewer_than_2_months['location_id'].nunique())))


In [ ]:
df_sales_all=pd.merge(df_sales_all,df_id_zip,on='customer_id_hashed',how="left")
df_sales_all['customer_zip_code']=df_sales_all['customer_zip_code'].fillna("nan")
df_sales_all['zip_validation']=np.where(pd.isnull(df_sales_all['DMA_set']),"Unknown_Zip","Valid_Zip")
# Stopped here



In [ ]:
df_sales_by_id=df_sales_all.groupby("customer_id_hashed")['sales'].sum().to_frame().reset_index()
df_sales_by_id=df_sales_by_id.sort_values("sales",ascending=False).reset_index()
del df_sales_by_id['index']
print(df_sales_by_id.shape)

for i in range(5,10):
    threshold=i*1000
    print(threshold,df_sales_by_id[df_sales_by_id['sales']>threshold].shape)

In [ ]:
output_folder="/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_"+str(datetime.datetime.now().date())+"/"

try:
    os.stat(output_folder)
except:
    os.mkdir(output_folder)

In [ ]:
# Stopped here

In [21]:
df_sales_by_id=df_sales_all.groupby("customer_id_hashed")['sales'].sum().to_frame().reset_index()
df_sales_by_id=df_sales_by_id.sort_values("sales",ascending=False).reset_index()
del df_sales_by_id['index']
print(df_sales_by_id.shape)

for i in range(5,10):
    threshold=i*1000
    print(threshold,df_sales_by_id[df_sales_by_id['sales']>threshold].shape)

(17883899, 2)
5000 (2711, 2)
6000 (1669, 2)
7000 (1196, 2)
8000 (910, 2)
9000 (726, 2)


In [22]:
# Sales >7000 ids removed, 1196 ids out of 17883899 rewards shoppers in total
id_list_removed=df_sales_by_id[df_sales_by_id['sales']>7000]['customer_id_hashed'].unique().tolist()

# df_sales_by_id=df_sales_by_id[~df_sales_by_id['customer_id_hashed'].isin(id_list_removed)]
df_sales_all=df_sales_all[~df_sales_all['customer_id_hashed'].isin(id_list_removed)]
del df_sales_by_id
import gc
gc.collect()

298

In [23]:
agg_func={"sales":"sum","trans":"sum","customer_id_hashed":"nunique"}
output_by_store_zip_sales=df_sales_all.groupby(['location_id','customer_zip_code','zip_validation'])['sales','trans','customer_id_hashed'].agg(agg_func).reset_index()
print(output_by_store_zip_sales.shape)
output_by_store_zip_sales.head(4)

(1811292, 6)


,location_id,customer_zip_code,zip_validation,sales,trans,customer_id_hashed
0,1,00000,Unknown_Zip,11558.01,177.0,59
1,1,02703,Valid_Zip,487.43,10.0,1
2,1,03256,Valid_Zip,51.67,2.0,1
3,1,04210,Valid_Zip,98.68,3.0,1


In [24]:
output_by_store_zip_sales.groupby("zip_validation")['customer_id_hashed'].sum().to_frame()

,customer_id_hashed
zip_validation,
Unknown_Zip,2727503
Valid_Zip,20479171


In [1]:
2727503/(20479171+2727503)

0.11753097406375425

In [26]:
output_by_store_zip_sales[output_by_store_zip_sales['location_id']!="6990"].groupby("zip_validation")['customer_id_hashed'].sum().to_frame()

,customer_id_hashed
zip_validation,
Unknown_Zip,2690162
Valid_Zip,20337889


In [2]:
2690162/(2690162+20337889)

0.116821089201166

In [28]:
output_by_store_zip_sales.shape

(1811292, 6)

In [29]:
output_by_store_zip_sales_cum=pd.DataFrame()
output_by_store_zip_sales=output_by_store_zip_sales.sort_values(["location_id",'sales'],ascending=[True,False])
for store,group in output_by_store_zip_sales.groupby("location_id"):
    group['cum_sales']=group['sales'].cumsum()
    total_sales=group['sales'].sum()
    group['pctg']=group['cum_sales']/total_sales
    group['revenue_flag']=np.where(group['pctg']>=0.8,"T",
                                  np.where(group['pctg']>=0.6,"S","P"))
    output_by_store_zip_sales_cum=output_by_store_zip_sales_cum.append(group)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [30]:
output_by_store_zip_trans_cum=pd.DataFrame()
output_by_store_zip_trans=output_by_store_zip_sales.sort_values(["location_id",'trans'],ascending=[True,False])
for store,group in output_by_store_zip_trans.groupby("location_id"):
    group['cum_trans']=group['trans'].cumsum()
    total_trans=group['trans'].sum()
    group['pctg']=group['cum_trans']/total_trans
    group['transactions_flag']=np.where(group['pctg']>=0.8,"T",
                                  np.where(group['pctg']>=0.6,"S","P"))
    output_by_store_zip_trans_cum=output_by_store_zip_trans_cum.append(group)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [31]:
output_by_store_zip_sales_cum=output_by_store_zip_sales_cum.rename(columns={"pctg":"sales_pctg_cum"})
output_by_store_zip_trans_cum=output_by_store_zip_trans_cum.rename(columns={"pctg":"trans_pctg_cum"})

output_by_store_zip_trans_label=output_by_store_zip_trans_cum[['location_id','customer_zip_code','cum_trans','trans_pctg_cum','transactions_flag']]
output_by_store_zip=pd.merge(output_by_store_zip_sales_cum,output_by_store_zip_trans_label,on=['location_id','customer_zip_code'],how="outer")

output_by_store_zip.head(2)

,location_id,customer_zip_code,zip_validation,sales,trans,customer_id_hashed,cum_sales,sales_pctg_cum,revenue_flag,cum_trans,trans_pctg_cum,transactions_flag
0,1,43232,Valid_Zip,602768.34,16101.0,3245,602768.34,0.214983,P,16101.0,0.260846,P
1,1,43207,Valid_Zip,348158.20,6688.0,1798,950926.54,0.339157,P,30159.0,0.488595,P


In [32]:
os.getcwd()

'/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906'

In [33]:
output_by_store_zip=pd.merge(output_by_store_zip,df_summary_store_date,on="location_id")

In [34]:
output_by_store_zip.to_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_by_store_zip_sales_cum_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [35]:
summary=output_by_store_zip.groupby(['location_id','revenue_flag','transactions_flag'])['customer_zip_code'].count().to_frame().reset_index()
summary=pd.merge(summary,df_summary_store_date,on="location_id",how="left")
summary.to_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_zip_count_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [36]:
summary_2=output_by_store_zip.groupby(['location_id','revenue_flag','transactions_flag','zip_validation'])['customer_zip_code'].count().to_frame().reset_index()
summary_2=pd.merge(summary_2,df_summary_store_date,on="location_id",how="left")
summary_2.to_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_zip_count_summary2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [37]:
output_by_store_zip_sameple_store_1=output_by_store_zip[output_by_store_zip['location_id']=="1"]
output_by_store_zip_sameple_store_1.to_csv("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/output_by_store_zip_sample_store_1_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [38]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/BL_new_ta_zip_by_store_summary_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary.to_excel(writer,"summary_by_store",index=False)
summary_2.to_excel(writer,"summary_2_by_store",index=False)
output_by_store_zip_sameple_store_1.to_excel(writer,"sample_store_1",index=False)
writer.save()